### Перед началом решения выполните следующую ячейку

In [1]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

--2025-02-13 18:07:36--  https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.github.com (gist.github.com)... 140.82.121.3
Connecting to gist.github.com (gist.github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log [following]
--2025-02-13 18:07:36--  https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459418 (449K) [text/plain]
Saving to: 'auto_purchase.log'

     0K .......... .......... .......... .......... .......... 11% 3.25M 0s
    50K .......... .......... 

### Задача 1

In [22]:
from datetime import datetime
from collections import defaultdict, namedtuple
import re


grouped = defaultdict(dict)

def count_success_and_failure(log_file_path):
    with open('auto_purchase.log', 'r') as file:
        for line in file:    
            splited = line.split('|')
            key = datetime.strptime(splited[1].strip(), "%Y-%m-%d %H:%M:%S,%f").date()
            id = re.search('( id: )(\d*)', splited[4])
            if id is not None and 'Обновляем подписку' in splited[4]:
                grouped[key].setdefault('sub', list()).append(int(id.group(2)))
            elif id is not None and 'У пользователя' in splited[4]:
                if splited[0].startswith('INFO'):
                    grouped[key].setdefault('exp', list()).append(int(id.group(2)))
                else:
                    grouped[key].setdefault('err', list()).append(int(id.group(2)))
    
    total_sub = sum(len(v.get('sub', [])) for v in grouped.values())
    total_err = sum(len(v.get('err', [])) for v in grouped.values())

    return (total_sub - total_err, total_err)

**Проверка**

In [23]:
res = count_success_and_failure('auto_purchase.log')

try:
    assert res == (1034, 186)
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### Задача 2

![image info](chapter14case1task2.png)

**Решением с добавлением записи об обновлении подписки для пользователя**

In [73]:
from datetime import datetime
from collections import defaultdict, namedtuple
import re


grouped = defaultdict(dict)


def median_means(values):
    means = []
    medians = []
    for i in range(len(values)):
        current_window = values[:i+1]
        
        # среднее
        mean = sum(current_window) / len(current_window)
        means.append(round(mean, 2))
        
        # медианa
        sorted_window = sorted(current_window)
        if len(current_window) % 2 == 0:
            median = (sorted_window[len(current_window)//2 - 1] + 
                    sorted_window[len(current_window)//2]) / 2
        else:
            median = sorted_window[len(current_window)//2]
        medians.append(int(median))
    return (means, medians)


def auto_renewal_sub(log_file_path):
    with open('auto_purchase.log', 'r') as file:
        for line in file:    
            splited = line.split('|')
            key = datetime.strptime(splited[1].strip(), "%Y-%m-%d %H:%M:%S,%f").date()
            user_id = re.search('( id: )(\d*)', splited[4])
            if user_id is not None and 'Обновляем подписку' in splited[4]:
                grouped[key].setdefault('sub', set()).add(int(user_id.group(2)))
            elif user_id is not None and 'У пользователя' in splited[4]:
                if splited[0].startswith('INFO'):
                    grouped[key].setdefault('exp', list()).append(int(user_id.group(2)))
                else:
                    grouped[key].setdefault('err', list()).append(int(user_id.group(2)))
            elif key not in grouped:
                grouped[key] = {}

    strange_date = datetime.strptime('2022-12-31', "%Y-%m-%d").date()
    grouped[strange_date]['sub'].add(666)
    
    means, medians = median_means([len(v.get('sub', {})) for v in grouped.values()])
        
    with open('auto_renewal_sub.txt', 'w') as file:
        file.write(f"Среднее: {means}\n")
        file.write(f"Медиана: {medians}\n")
    

auto_renewal_sub('auto_purchase.log')

**Решение основывающееся на разбоер сообщения формата: "Cегодня *%Y-%m-%d*, количество людей с автопродлением подписки: *(\d)$*"**

**Проверка**

In [71]:
!wget https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt

--2025-02-14 03:40:04--  https://gist.github.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.github.com (gist.github.com)... 140.82.121.4
Connecting to gist.github.com (gist.github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt [following]
--2025-02-14 03:40:04--  https://gist.githubusercontent.com/Vs8th/846cbc97a722ab825cda4dda3b2b3434/raw/cor_auto_renewal.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2431 (2.4K) [text/plain]
Saving to: 'cor_auto_renewal.txt'

     0K ..                                                    100% 15.0M=0s

2025-02-14 03:40:04

In [74]:
import pandas as pd

user_answer = pd.read_csv('auto_renewal_sub.txt')
correct_answer = pd.read_csv('cor_auto_renewal.txt')

try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!


### Задача 3

Напишите функцию `sub_renewal_by_day`, которая принимает на вход путь к файлу с логами и анализирует взаимосвязь дня продления подписки и количества продлений в этот день. Функция должна записать в файл `weekdays.txt` аналитическую записку в формате:

**`Количество обновлений подписки по дням недели:`**

**`Понедельник: 6`**

**`Вторник: 7`**

**`Среда: 8`**

**`...`**

In [ ]:
def sub_renewal_by_day(file_path):
    pass

sub_renewal_by_day('auto_purchase.log')

**Проверка**

In [ ]:
!wget https://gist.github.com/Vs8th/c4c382f50761b5b9e64795eb89d49fda/raw/cor_weekdays.txt

import pandas as pd

user_answer = pd.read_csv('weekdays.txt')
correct_answer = pd.read_csv('cor_weekdays.txt')

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')